In [77]:
import cv2 as cv
import os
import numpy as np
#import tracker1 as tr
import tracker1
#rom tracker1 import Objecte as o

In [197]:
#tracker

class Tracker():
    
    def __init__(self, objectes = [], ids = []):
        self._objectes = []
        self._ids = []
        


    def deteccio(self, objectes_detectats):
        
        if self._objectes == []:
            for i, new_object in enumerate(objectes_detectats):              
                id = len(self._ids) + 1
                new_object._id = id
                
                self._ids.append(id)
                self._objectes.append(new_object)
        
        
        else:
            matriu_distancies = np.zeros( (len(self._objectes), len(objectes_detectats)))
            
            for i, objecte in enumerate(self._objectes):
                for j, objecte_candidat in enumerate(objectes_detectats):
                    matriu_distancies[i,j] = objecte.dist_euclidean(objecte_candidat)
            
            
            valor_maximo = matriu_distancies.sum()
            
            if len(self._objectes) <= len(objectes_detectats):
                for i in range (len(self._objectes)):
                    ix_objectes, ix_detactats = np.where(matriu_distancies == np.min(matriu_distancies))
                    ix_objectes, ix_detactats = ix_objectes[-1], ix_detactats[-1]
                    
                    
                    self._objectes[int(ix_objectes)]._centroide = objectes_detectats[int(ix_detactats)]._centroide
                    
                    
                    matriu_distancies[ix_objectes, :] = valor_maximo
                    matriu_distancies[:, ix_detactats] = valor_maximo
                    
                for i in range(len(objectes_detectats) - len(self._objectes) ):
                    ix_objectes, ix_detactats = np.where(matriu_distancies == np.min(matriu_distancies))
                    
                    new_object = objectes_detectats[int(ix_detactats[-1])]
                    id = len(self._ids) + 1
                    new_object._id = id
                    self._objectes.append(new_object)
                    self._ids.append(id)
                    
                    matriu_distancies[ix_objectes, :] = valor_maximo
                    matriu_distancies[:, ix_detactats] = valor_maximo
            
                
            
            
            
class Objecte():
    
    def __init__(self, centroide = (0,0), id = None):
        self._centroide = centroide
        self._id = id
        
    
    def dist_euclidean(self, altre_centroide):
        dist = (self._centroide[0] - altre_centroide._centroide[0]) ** 2 + (self._centroide[1] - altre_centroide._centroide[1]) ** 2
        return dist


In [138]:
tracking = Tracker()


In [54]:
capture = cv.VideoCapture('output7.mp4')
count = 0
path = 'C:/Users/LENOVO/Documents/uni/4t/psiv/tracking/psiv_trackers/frames/'
while ( capture.isOpened()):
    ret, frame = capture.read()
    if (ret == True):
        cv.imwrite(path +  'img_%010d.jpg' % count, frame)
        count += 1
        if cv.waitKey(1) == ord('s'):
            break
    else:
        break
capture.release()
cv.destroyAllWindows()

#guardar imagene en un archivo

KeyboardInterrupt: 

In [206]:
capture = cv.VideoCapture('output7.mp4')
fgbg = cv.bgsegm.createBackgroundSubtractorMOG()

tracking = Tracker()

print(tracking._ids)

while ( capture.isOpened()):
    ret, frame = capture.read()
    if (ret == False): break
    
    
    altura, ancho, _ = frame.shape
    
    
    roi = frame[400:, :]
    
    
    fgmaskMOG = fgbg.apply(roi)
     
    tratada = cv.dilate(fgmaskMOG, np.ones((3,3)), iterations=7)
    tratada = cv.morphologyEx(tratada,cv.MORPH_OPEN, np.ones((30,30)))
    
    
    #detectar objeto
    objetetos_detectados = []
    contours, _ = cv.findContours(tratada, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area = cv.contourArea(cnt)
        if area > 4000:
        #Calculate area remove small elements (personas/motos)
            #cv.drawContours(roi, [cnt], -1, (0, 255, 0), 2)
            x, y, w, h = cv.boundingRect(cnt)
            
            if 0.6 < w/h and   w/h < 1.5:
                cv.rectangle(roi, (x,y), (x + w, y + h), (0, 255, 0), 3)
                
                centroide = (int(x + w/2) ,int(y + h/2))
                
                objecte = Objecte(centroide)
                objetetos_detectados.append(objecte)
            
            tracking.deteccio(objetetos_detectados)
            
            for ob in tracking._objectes:
                cv.putText(roi, str(ob._id), ob._centroide, cv.FONT_HERSHEY_SIMPLEX, 0.3, (2255, 255, 0) )
            
        
    
    cv.imshow('roi', roi)
    cv.imshow('MOG', tratada)
    cv.imshow('frame', frame)
    
    
    if cv.waitKey(1) == ord('q'):
            break
    
    
capture.release()
cv.destroyAllWindows()


[]


KeyboardInterrupt: 

In [182]:
capture.release()
cv.destroyAllWindows()